## Setup Docker

Install docker engine for Ubuntu using the convenience script > Post-installation steps.

Install remote development extension in VSCode.

## Cuda-Pytorch
- Install and configure nvidia container toolkit
- Add "--gpus","all" in "runArgs"
  "runArgs": [
    "--privileged",
    "--network=host",
    "--gpus","all"
  ],
- Install correct cuda for latest PyTorch (default cuda may be fine), install default Pytorch
- Don't install Python/Anaconda

# Docker image tries

## my_bot
Disconnect VPN

{
  "name": "humble desktop-full",
  "dockerFile": "Dockerfile",
  "runArgs": [
    "--privileged",
    "--network=host",
    "--gpus","all"
  ],
  "workspaceMount": "source=${localWorkspaceFolder},target=/${localWorkspaceFolderBasename},type=bind",
  "workspaceFolder": "/${localWorkspaceFolderBasename}",
  "mounts": [
    "source=${localEnv:HOME}${localEnv:USERPROFILE}/.bash_history,target=/home/vscode/.bash_history,type=bind"
],
"features": {
	"ghcr.io/devcontainers/features/desktop-lite:1": {},
  "ghcr.io/devcontainers/features/nvidia-cuda:1": {},
  "ghcr.io/raucha/devcontainer-features/pytorch:1": {}
}
}


## Articulated Robotics: Make a Mobile Robot

Differential Drive: robot with two wheels driving it. 

Base link: main coordinate system: x-pointing forward, y-pointing left, z-pointing up.

![Robot State Publisher](./images/Robot_state_publisher.png)

#### Running GUI
xhost +local:  
In terminal, and rebuilding container.

```bash
cd my_bot && mkdir src
git clone https://github.com/SunzidHassan/my_bot.git
cd my_bot
- pip install catkin_pkg
colcon build --symlink-install
```

#### Building Xacro robot description
Everytime we add a new file:  
```bash
colcon build --symlink-install
```

#### State publisher
Then we need to source the updated install in state publisher, gazebo, etc.:  
```bash
source install/setup.bash
```

State publisher: 
```bash
ros2 launch my_bot rsp.launch.py use_sim_time:=true
```

#### Install Gazebo
```bash
sudo apt update && sudo apt upgrade -y
sudo apt install ros-humble-gazebo-ros-pkgs
```

#### Gazebo simulation
```bash
rosdep update
```

Gazebo launch:  
```bash
ros2 launch gazebo_ros gazebo.launch.py  
```

Launching with launcher file:  
```bash
ros2 launch my_bot launch_sim.launch.py
```

If the normal or 3d camera is not working, try
- colcon build and source setup.bash.
- starting from a fresh gazebo env,
- install python

Launching with launcher in custom simulation environment:  
```bash
ros2 launch my_bot launch_sim.launch.py world:=./src/my_bot/worlds/football.world 
```

#### Rviz load view
```bash
rviz2 -d src/my_bot/config/view_bot.rviz
```
#### Teleop
```bash
ros2 run teleop_twist_keyboard teleop_twist_keyboard
```

![Robot Control](.images/robot_control_1.png)

### Camera
```bash
ros2 run rqt_image_view rqt_image_view

ros2 run image_transport republish compressed raw --ros-args -r in/compressed:=/camera/image_raw/compressed -r out:=/camera/image_raw/uncompressed
```

# Building Mobile Robot

## Describing robot with unified robot description format (URDF)
### `Links`
- Seperate components, or
- Can move indepedently

**Link coordinate system**: if the `link` can rotate, it should have the origin at the pivot point. Example: 4 link coordinate system for: `Base`, `Slider`, `Arm`, `Camera`.

**Link joints**: joints show relation of the origin and coordinate frame of the links, which'll determine the position and orientation of the links.
It also shows what other link it's connected to - the `Parent`, and how it's connected to the `Parent`. Each link can have 1 parent, but many child links, each with own joints.
Example: a joint from `Base` to the `Slider`, one from the `Slider` to the `Arm`, and one from the `Arm` to the `Camera`.

**Link motion**:
- *Revolute*: rotational motion about a point, with fixed start and stop angle.
- *Continuous*: rotational motion about a point, without fixed limit - wheel.
- *Prismatic*: linear translational motion.
- *Fixed*: child link doesn't move relative to the parent link.

### URDF syntax
URDF is based on XML, where everything is represented as a series of tags nested inside each other.

```XML
<?xml version="1.0"?>
<robot name="my_bot">
<link name="link_name"></link>
<joint name="joint_name"></joint>
<link></link>
...
</robot>
```

#### Links
Each link tag will represent 1 link. It must have a name, but can also have 3 additional properties:
- <visual>
    - <geometry> defines the overall shape (box, cylinder, sphere, path to a 3d mesh)
    - <origin> offset for the geometry, so that it doesn't have to center around the origin.
    - <material> for color
</visual>

- <collision> for physics calculations, includes <geometry> and <origin> that can be copied from <visual>. But this can be made simple for computational savings.
    - <geometry> defines the overall shape (box, cylinder, sphere, path to a 3d mesh)
    - <origin> offset for the geometry, so that it doesn't have to center around the origin.
</collision>

- <inertial> also for physics calculation - how the links react to forces.
    - <mass>
    - <origin> the center of gravity, for most case the same as visual and collision origin.
    - <inertia> rotational inertia matrix. We can approximate our links as prisms, ellipsoids or cylinders.
</inertial>

#### Joints
Joints specify where the links are in space.
Parent and child links define links this joint 

```xml
<joint name="arm_joint" type="revolute">
    <parent link="slider_link"/>
    <child link="arm_link"/>
    <origin xyz="0.25 0 0.15" rpy="0 0 0"> relationship of the links before any motion is applied.
    For non-fixed joints, we need at least two more characteristics:
    <axis xyz="0 -1 0"/> which axes (combination of x, y, z) the joint moves along or aournd.
    <limits lower="0" upper="${pi/2}" velocity="100" effort="100"/> upper and lower positional limits in radians or meters, velocity limit in radians or meters per second, and effort limits in newtons or newton meters.
</joint>
```

Other tags include <material> tag for defining colors, <gazebo> tag for simulation, <transmission> tag that defines how an actuator is connected to a joint.

Naming convention: "_link", "_joint". Like "arm_link" is the child link in "arm_joint".

#### xacro
XML macro (xacro) helps with urdf files.
- Splitting urdf into multiple files
- Avoidance of repetition

To add xacro in a URDF file:
<robot xmlns:xacro="http://www.ros.org/wiki/xacro">

xacro compiles split urdf files into one urdf, which is fed to robot_state_publisher, which publishes the complete urdf on /robot_description topic.

my_robot.urdf.xacro
'''xml
<?xml version="1.0"?>
<robot xmlns:xacro="http://www.ros.org/wiki/xacro" name="my_robot">
    <xacro:include filename="my_materials.xacro"/>
    <link>...</link>
    <joint>...</joint>
</robot>
'''

my_materials.xacro
'''xml
<?xml version="1.0"?>
<robot xmlns:xacro="http://www.ros.org/wiki/xacro">
    <material name="white">
        <color rgba="1 1 1 1"/>
    </material>
</robot>
'''

*Properties*: like setting variables.
<xacro:property name="arm_radius" value="0.5"/>
...
<cylinder radius="${arm_radius}" length="7"/>

*Math operations*:
<cylinder length="${4*arm_radius*pi1}"/>

*Macros*:
<xacro:macro name="inertial_box" params="mass x y z *origin">
    <inertial1>
        <xacro:insert_block name="origin"/>
        <mass value="${mass}"/>
        <inertia ixx="${(1/12) * mass * (y*y+z*z)}" *other entries skipped*/>
    </inertial>
</xacro:macro>

User later:
<xacro:inertial_box mass="12" x="2" y="3" z="4">
    <origin xyz="0 2 4" rpy="0 0 0"/>
</xacro:inertial_box>

will expand to:
<inertial>
    <origin xyz="0 2 4" rpy="0 0 0"/>
    <mass value="12"/>
    <inertia ixx="25"/>
</inertial>

#### Example

## Creating rough 3D model

Robot link: base_link
Orientation: x-forward, y-left, z-up


# Rough works

In [ ]:
FROM osrf/ros:humble-desktop-full

# Add vscode user with same UID and GID as your host system
# (copied from https://code.visualstudio.com/remote/advancedcontainers/add-nonroot-user#_creating-a-nonroot-user)
ARG USERNAME=vscode
ARG USER_UID=1000
ARG USER_GID=$USER_UID
RUN groupadd --gid $USER_GID $USERNAME \
    && useradd -s /bin/bash --uid $USER_UID --gid $USER_GID -m $USERNAME \
    && apt-get update \
    && apt-get install -y sudo \
    && echo $USERNAME ALL=\(root\) NOPASSWD:ALL > /etc/sudoers.d/$USERNAME \
    && chmod 0440 /etc/sudoers.d/$USERNAME
# Switch from root to user
USER $USERNAME

# Add user to video group to allow access to webcam
RUN sudo usermod --append --groups video $USERNAME

# Update all packages
RUN sudo apt update && sudo apt upgrade -y

# Install Git
RUN sudo apt install -y git

# Rosdep update
RUN rosdep update

# Source the ROS setup file
RUN echo "source /opt/ros/${ROS_DISTRO}/setup.bash" >> ~/.bashrc

# Install wget
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt-get install -y wget

# Add workspace directory
RUN cd my_bot && mkdir src
RUN git clone https://github.com/SunzidHassan/my_bot.git
RUN cd my_bot
RUN pip install catkin_pkg
RUN colcon build --symlink-install

# Install gazebo
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-gazebo-ros-pkgs


## Turtlebot4

In [ ]:
#  Command to build it
# docker built -t <image name > .
FROM osrf/ros:humble-desktop-full

# Add vscode user with same UID and GID as your host system
# (copied from https://code.visualstudio.com/remote/advancedcontainers/add-nonroot-user#_creating-a-nonroot-user)
ARG USERNAME=vscode
ARG USER_UID=1000
ARG USER_GID=$USER_UID
RUN groupadd --gid $USER_GID $USERNAME \
    && useradd -s /bin/bash --uid $USER_UID --gid $USER_GID -m $USERNAME \
    && apt-get update \
    && apt-get install -y sudo \
    && echo $USERNAME ALL=\(root\) NOPASSWD:ALL > /etc/sudoers.d/$USERNAME \
    && chmod 0440 /etc/sudoers.d/$USERNAME
# Switch from root to user
USER $USERNAME

# Add user to video group to allow access to webcam
RUN sudo usermod --append --groups video $USERNAME

# Update all packages
RUN sudo apt update && sudo apt upgrade -y

# Install Git
RUN sudo apt install -y git && apt-get install -y python3-pip

# Rosdep update (disconnect vpn)
RUN rosdep update

RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-dev-tools

# Install wget
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt-get install -y wget

# Install Ingition
RUN sudo sh -c 'echo "deb http://packages.osrfoundation.org/gazebo/ubuntu-stable `lsb_release -cs` main" > /etc/apt/sources.list.d/gazebo-stable.list'
RUN wget http://packages.osrfoundation.org/gazebo.key -O - | sudo apt-key add -
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt-get install -y ignition-fortress

# Install simulator
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-turtlebot4-simulator

# Add workspace directory
RUN mkdir -p ~/turtlebot4_ws/src && cd ~/turtlebot4_ws/src/

RUN git clone https://github.com/turtlebot/turtlebot4_simulator.git -b humble

RUN cd ..

RUN sudo rosdep init
RUN rosdep update

RUN rosdep install -y --from-path src -yi

# Install catkin_pkg before colcon build
RUN sudo apt update && sudo apt upgrade -y
RUN pip install catkin_pkg

# Source the ROS setup file
RUN echo "source /opt/ros/humble/setup.bash" >> ~/.bashrc

RUN colcon build --symlink-install


RUN echo "ALL Done"

## Turtlebot3

In [ ]:
#  Command to build it
# docker built -t <image name > .
FROM osrf/ros:humble-desktop-full

# Add vscode user with same UID and GID as your host system
# (copied from https://code.visualstudio.com/remote/advancedcontainers/add-nonroot-user#_creating-a-nonroot-user)
ARG USERNAME=vscode
ARG USER_UID=1000
ARG USER_GID=$USER_UID
RUN groupadd --gid $USER_GID $USERNAME \
    && useradd -s /bin/bash --uid $USER_UID --gid $USER_GID -m $USERNAME \
    && apt-get update \
    && apt-get install -y sudo \
    && echo $USERNAME ALL=\(root\) NOPASSWD:ALL > /etc/sudoers.d/$USERNAME \
    && chmod 0440 /etc/sudoers.d/$USERNAME
# Switch from root to user
USER $USERNAME

# Add user to video group to allow access to webcam
RUN sudo usermod --append --groups video $USERNAME

# Update all packages
RUN sudo apt update && sudo apt upgrade -y

# Install Git
RUN sudo apt install -y git

RUN sudo apt install -y python3

# Rosdep update
RUN rosdep update

RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-dev-tools

# Install wget
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt-get install -y wget

############################################################
## Turtlebot3 Simulation
# Install Ingition
RUN sudo sh -c 'echo "deb http://packages.osrfoundation.org/gazebo/ubuntu-stable `lsb_release -cs` main" > /etc/apt/sources.list.d/gazebo-stable.list'
RUN wget http://packages.osrfoundation.org/gazebo.key -O - | sudo apt-key add -
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt-get install -y ignition-fortress

# Install simulator
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-turtlebot4-simulator

# Add workspace directory
RUN mkdir -p ~/turtlebot4_ws/src && cd ~/turtlebot4_ws/src/

RUN git clone https://github.com/turtlebot/turtlebot4_simulator.git -b humble

RUN cd ..

RUN sudo rosdep init
RUN rosdep update

RUN rosdep install -y --from-path src -yi

# Install catkin_pkg before colcon build
RUN sudo apt update && sudo apt upgrade -y
RUN pip install catkin_pkg

# Source the ROS setup file
RUN echo "source /opt/ros/humble/setup.bash" >> ~/.bashrc

RUN colcon build --symlink-install


############################################################
## Turtlebot3 Simulation

# Gazebo install
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-gazebo-*

# Cartographer
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-cartographer
RUN sudo apt install -y ros-humble-cartographer-ros

# Navigation 2
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-navigation2
RUN sudo apt install -y ros-humble-nav2-bringup

# Turtlebot3 packages
RUN sudo apt update && sudo apt upgrade -y
RUN source ~/.bashrc
RUN sudo apt install -y ros-humble-dynamixel-sdk
RUN sudo apt install -y ros-humble-turtlebot3-msgs
RUN sudo apt install -y ros-humble-turtlebot3

# Environment configuration
RUN sudo apt update && sudo apt upgrade -y
RUN echo 'export ROS_DOMAIN_ID=30 #TURTLEBOT3' >> ~/.bashrc
RUN source ~/.bashrc

# Turtlebot3 simulation
RUN cd ~/turtlebot3/src
RUN git clone -b humble-devel https://github.com/ROBOTIS-GIT/turtlebot3_simulations.git
RUN cd ~/turtlebot3_ws && colcon build --symlink-install


RUN echo "ALL Done"

Build the dockerfile and get Docker image ID: docker build -t r2_humble_turtlebot34 .

make the bash file and Dockerfile executable

./run_docker_gpu.bash

In [ ]:
xhost local:root

XAUTH=/tmp/.docker.xauth

docker run -it \
    --name=turtlebot4_simulator_container \
    --env="DISPLAY=$DISPLAY" \
    --env="QT_X11_NO_MITSHM=1" \
    --volume="/tmp/.X11-unix:/tmp/.X11-unix:rw" \
    --env="XAUTHORITY=$XAUTH" \
    --volume="$XAUTH:$XAUTH" \
    --net=host \
    --privileged \
    --runtime=nvidia \
    r2_humble_turtlebot34 \
    bash

echo "Done."